In [2]:
import sys
sys.path.append('..')

from langchain.vectorstores import FAISS
import glob
import yaml
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings

def load_documents(folder_path):
    documents = []
    file_paths = glob.glob(folder_path + "/*.yaml")
    
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            document = yaml.safe_load(file)
            documents.append(document)
    
    return documents

def create_vector_store(folder_path, vector_store_path, embeddings):
    # Load documents from the folder path
    documents = load_documents(folder_path)
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(vector_store_path)
    
    print("Vector store created and saved successfully!")

def load_vector_store(vector_store_path, embeddings):
    # Load the vector store from the local file system
    vectorstore = FAISS.load_local(vector_store_path, embeddings)
    
    print("Vector store loaded successfully!")
    
    return vectorstore




In [3]:
from dotenv import load_dotenv

def load_env_variables(file_path):
    load_dotenv(file_path)
    print("Environment variables loaded successfully!")

env_file_path = "../.env"
load_env_variables = load_env_variables(env_file_path)

# Rest of the code
embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002",
                                   openai_api_version="2023-03-15-preview")

Environment variables loaded successfully!


/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://polariseastus2.openai.azure.com/ to https://polariseastus2.openai.azure.com//openai.
  warnings.warn(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py

In [3]:
folder_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/dummy_model_files"
vector_store_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vectors"
create_vector_store(folder_path, vector_store_path, embeddings)

NameError: name 'embeddings' is not defined

In [4]:
from langchain import PromptTemplate
start_goal_prompt = PromptTemplate(
    template="""
    You are an very experienced and expert manager. You are given following query "{goal}".
    The query may contain different objectives which needs an expertise to solve. Your job is to provide a list of objectives that is easier for the experts to serve the query.
    Ensure objectives are streamlined to minimize the overall number of tasks while addressing the objective effectively. 
    Some part of the query may not be relevant to the objectives. You can ignore those parts.
    The number of tasks should be strictly less than 10.
    """,
    input_variables=["goal", "language", "tools"],
)

Tools

Metric Discovery: Assuming we are trying to 3 metrics 
Agent: Get relevant 
Get 3 top documents 

Tasks: Metric Discovery 
1. Rewrite the and Extract specifics of metric information for better retrieval
2. Extract top 3 documents related to metric information.
3. get information about metrics and compile the results and return 



In [6]:
import json
import os
from langchain.chat_models import AzureChatOpenAI


llm_gpt = AzureChatOpenAI(deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'), openai_api_version=os.getenv("OPENAI_API_VERSION"),
                        openai_api_base=os.getenv("OPENAI_API_BASE"), 
                        openai_api_type= os.getenv("OPENAI_API_TYPE"),
                        openai_api_key=os.getenv("OPENAI_API_KEY"),
                        max_tokens=3500,
                        temperature=0.0)
# for key, value in os.environ.items():
#   print(f"{key}: {value}")

from crewai import Agent, Task
from langchain.tools import tool

class ExtractMetricInfo():
  @tool("Extract metric queried.")
  def extract_metric_info(query):
    """Useful to isolate metric information to be further used in the its discovery"""
    agent = Agent(
          role='Metric Isolator',
          goal=
          'Accurate Extraction of Metrics name',
          backstory=
          "You're a Metric isolator which is responsible for extracting the metrics name from the given content.",
          llm = llm_gpt,
          allow_delegation=False)
    task = Task(
          agent=agent,
          description=
          f' The primary objective is to ensure that all the metrics are correctly isolated and extracted from the query {query}. This extracted information should be organized in a form of list.',
          expected_output="some string",
    
      )
    extracted_metrics = task.execute()

    return extracted_metrics

/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://polariseastus2.openai.azure.com/ to https://polariseastus2.openai.azure.com/openai.
  warnings.warn(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/chat_models/azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is spec

In [7]:
query = "What is the most popular feature used by our paid subscribers"
extracted_metrics = ExtractMetricInfo.extract_metric_info(query)

In [8]:
extracted_metrics 

'"feature popularity among paid subscribers"'